## Q1

In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext, SQLContext


appName = "Big Data Analytics"
master = "local"

# Create Configuration object for Spark.
conf = pyspark.SparkConf()\
.set('spark.driver.host','127.0.0.1')\
.setAppName(appName)\
.setMaster(master)

# Create Spark Context 
sc = SparkContext.getOrCreate(conf=conf)

# Create SQL Context to conduct some database operations
sqlContext = SQLContext(sc)

# If you have SQL context, you create the session from the Spark Context
spark = sqlContext.sparkSession.builder\
.config("spark.jars.packages", "org.postgresql:postgresql:42.2.29") \
.config("spark.sql.execution.arrow.enabled","true")\
.getOrCreate() 

24/09/22 20:35:05 WARN Utils: Your hostname, Kitiyas-MacBook-Air-5.local resolves to a loopback address: 127.0.0.1; using 10.0.0.118 instead (on interface en0)
24/09/22 20:35:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/22 20:35:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/opt/anaconda3/envs/mlp/lib/python3.8/site-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(
24/09/22 20:35:07 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.
24/09/22 20:35:07 WARN SparkSession: Using an existing Spark session; only runtime S

In [2]:
# Import data into sparkDF
url = "/Users/kitiya/Documents/CMU@2024/14763 Systems&ToolChain/hw3/shopping_trends.csv"

col_names = [
    "customer_id",
    "age",
    "gender",
    "item_purchased",
    "category",
    "purchase_amount",
    "location",
    "size",
    "color",
    "season",
    "review_rating",
    "subscription_status",
    "payment_method",
    "shipping_type",
    "discount_applied",
    "promocode_used",
    "previous_purchases",
    "preferred_payment",
    "freq_of_purchase"]

sparkDF = spark.read.csv(url,header=True, inferSchema=True).toDF(*col_names)
sparkDF.describe().show()

24/09/22 20:35:15 WARN SQLConf: The SQL config 'spark.sql.execution.arrow.enabled' has been deprecated in Spark v3.0 and may be removed in the future. Use 'spark.sql.execution.arrow.pyspark.enabled' instead of it.
24/09/22 20:35:15 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+------------------+-----------------+------+--------------+-----------+------------------+--------+----+------+------+------------------+-------------------+--------------+--------------+----------------+--------------+------------------+-----------------+----------------+
|summary|       customer_id|              age|gender|item_purchased|   category|   purchase_amount|location|size| color|season|     review_rating|subscription_status|payment_method| shipping_type|discount_applied|promocode_used|previous_purchases|preferred_payment|freq_of_purchase|
+-------+------------------+-----------------+------+--------------+-----------+------------------+--------+----+------+------+------------------+-------------------+--------------+--------------+----------------+--------------+------------------+-----------------+----------------+
|  count|              3900|             3900|  3900|          3900|       3900|              3900|    3900|3900|  3900|  3900|              3900|     

Refer to q1_1.png, q1_2.png

## Q2

In [3]:
pip install neo4j

Note: you may need to restart the kernel to use updated packages.


In [233]:
from neo4j import GraphDatabase

# Replace with your actual Neo4j AuraDB credentials
url = "neo4j+s://c2cad06f.databases.neo4j.io"
user = "neo4j"
password = "SOcZdIr9LsT9ogaoikD2ei1njNpNTI9bXxDPyCA2Y2Y"

class Neo4JConnection:
    def __init__(self, url, user, password):
        """Initialize the Neo4j connection with URI, username, and password."""
        self.driver = GraphDatabase.driver(url, auth=(user, password))

    def close(self):
        """Close the Neo4j connection."""
        if self.driver:
            self.driver.close()

    def execute_query(self, query, parameters):
        """Execute a Cypher query and return the result."""
        with self.driver.session() as session:
            result = session.run(query, parameters)
            return result.data()

# Initialize Neo4j connection
neo4j_conn = Neo4JConnection(url, user, password)

In [14]:
# Function for deleting all nodes
def delete_all_nodes(connection):
    query = """
    MATCH (n)
    DETACH DELETE n;
    """
    connection.execute_query(query)
    print("All nodes deleted.")

In [103]:
delete_all_nodes(neo4j_conn)

/var/folders/5t/qhv6pkk115b3_mpr1bbc5c9c0000gn/T/ipykernel_90223/2864440883.py:20: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with self.driver.session() as session:


All nodes deleted.


In [104]:
# Create user node and item node with relationship
def create_nodes_and_relationships(neo4j_conn):
    for row in sparkDF.collect():
        query = """
        MERGE (u:User {customer_id: $id, age: $age, location: $location})
        MERGE (i:Item {item_purchased: $item})
        MERGE (u)-[p:PURCHASED {purchase_amount: $purchase_amount, season: $season, shipping_type: $shipping, review_rating: $review, discount_applied: $discount, promocode_used: $promo, previous_purchases: $prev_purchases, preferred_payment: $preferred_payment, freq_of_purchase: $freq_of_purchase}]->(i);
        """
        parameters = {
            'id': int(row['customer_id']),
            'age': int(row['age']),
            'location': row['location'],
            'item': row['item_purchased'],
            'purchase_amount': row['purchase_amount'],
            'season': row['season'],
            'shipping': row['shipping_type'],
            'review': row['review_rating'],
            'discount': row['discount_applied'],
            'promo': row['promocode_used'],
            'prev_purchases': row['previous_purchases'],
            'preferred_payment': row['preferred_payment'],
            'freq_of_purchase': row['freq_of_purchase']
        }
        neo4j_conn.execute_query(query, parameters)
    print("Nodes and relationships created.")


In [105]:
create_nodes_and_relationships(neo4j_conn)

/var/folders/5t/qhv6pkk115b3_mpr1bbc5c9c0000gn/T/ipykernel_90223/2864440883.py:20: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with self.driver.session() as session:


Nodes and relationships created.


In [183]:
# Query to retrieves the stored records
def check_nodes_and_relationships(neo4j_conn):
    # Check the User and Item nodes
    query_nodes = """
    MATCH (u:User)-[p:PURCHASED]->(i:Item)
    RETURN u.customer_id AS customer_id, u.age AS age, u.location AS location, 
           i.item_purchased AS item, 
           p.season AS season, p.shipping_type AS shipping, p.review_rating AS review;
    """
    
    result = neo4j_conn.execute_query(query_nodes,None)
    
    # Print the results
    print("Sample Nodes and Relationships:")
    for record in result:
        print(f"User ID: {record['customer_id']}, Age: {record['age']}, Location: {record['location']}")
        print(f"Item Purchased: {record['item']}")
        print(f"Relationship Detail: Season: {record['season']}, Shipping Type: {record['shipping']}, Review Rating: {record['review']}")
        print("-" * 30)

In [184]:
check_nodes_and_relationships(neo4j_conn)

Sample Nodes and Relationships:
User ID: 1, Age: 55, Location: Kentucky
Item Purchased: Blouse
Relationship Detail: Season: Winter, Shipping Type: Express, Review Rating: 3.1
------------------------------
User ID: 5, Age: 45, Location: Oregon
Item Purchased: Blouse
Relationship Detail: Season: Spring, Shipping Type: Free Shipping, Review Rating: 2.7
------------------------------
User ID: 36, Age: 54, Location: Massachusetts
Item Purchased: Blouse
Relationship Detail: Season: Summer, Shipping Type: 2-Day Shipping, Review Rating: 4.0
------------------------------
User ID: 51, Age: 49, Location: Maryland
Item Purchased: Blouse
Relationship Detail: Season: Spring, Shipping Type: Store Pickup, Review Rating: 3.7
------------------------------
User ID: 67, Age: 31, Location: Pennsylvania
Item Purchased: Blouse
Relationship Detail: Season: Winter, Shipping Type: 2-Day Shipping, Review Rating: 4.7
------------------------------
User ID: 137, Age: 39, Location: South Carolina
Item Purchased:

In [185]:
# Check items with unique nodes
def query_all_nodes(connection):
    query = """
    MATCH (n: Item)
    RETURN n;
    """
    result = connection.execute_query(query,None)
    print("All item nodes retrieved.")

    for i in result:
        print(i)

In [186]:
query_all_nodes(neo4j_conn)

All item nodes retrieved.
{'n': {'item_purchased': 'Blouse'}}
{'n': {'item_purchased': 'Sweater'}}
{'n': {'item_purchased': 'Jeans'}}
{'n': {'item_purchased': 'Sandals'}}
{'n': {'item_purchased': 'Sneakers'}}
{'n': {'item_purchased': 'Shirt'}}
{'n': {'item_purchased': 'Shorts'}}
{'n': {'item_purchased': 'Coat'}}
{'n': {'item_purchased': 'Handbag'}}
{'n': {'item_purchased': 'Shoes'}}
{'n': {'item_purchased': 'Dress'}}
{'n': {'item_purchased': 'Skirt'}}
{'n': {'item_purchased': 'Sunglasses'}}
{'n': {'item_purchased': 'Pants'}}
{'n': {'item_purchased': 'Jacket'}}
{'n': {'item_purchased': 'Hoodie'}}
{'n': {'item_purchased': 'Jewelry'}}
{'n': {'item_purchased': 'T-shirt'}}
{'n': {'item_purchased': 'Scarf'}}
{'n': {'item_purchased': 'Hat'}}
{'n': {'item_purchased': 'Socks'}}
{'n': {'item_purchased': 'Backpack'}}
{'n': {'item_purchased': 'Belt'}}
{'n': {'item_purchased': 'Boots'}}
{'n': {'item_purchased': 'Gloves'}}


Refer to q2.png

## Q3

In [187]:
# Find the overall percentage of users who are 50+ years old
def percentage_of_elder_user(connection):
    query = """
    MATCH (n:User)
    WITH COUNT(n) AS total
    MATCH (n:User)
    WHERE n.age >= 50
    WITH COUNT(n) AS elders, total
    RETURN elders * 1.0 / total * 100 AS percentage_elders;
    """
    result = connection.execute_query(query, None)
    print("the overall percentage of users who are 50+ years old: {} %".format(result[0]['percentage_elders']))

In [188]:
percentage_of_elder_user(neo4j_conn)

the overall percentage of users who are 50+ years old: 39.97435897435898 %


Refer to q3.png

## Q4

In [236]:
# Find the most purchased item in Hawaii
def most_purchased_Hawaii(connection):
    query = """
        MATCH (u:User)-[p:PURCHASED]-(i:Item)
        WHERE u.location = 'Hawaii'
        WITH COUNT(p.purchase_amount) as amount, i
        RETURN i as item 
        ORDER BY amount DESC
        LIMIT 1;
    """
    result = connection.execute_query(query,None)
    print("The most purchased item in Hawaii: ",result)

In [237]:
most_purchased_Hawaii(neo4j_conn)

The most purchased item in Hawaii:  [{'item': {'item_purchased': 'Handbag'}}]


Refer to q4.png

## Q5

In [197]:
# Find the most popular season to shop in and the most popular shipping method in this season
def most_popular_season_and_shipping(connection):
    # First, query the most popular season then save as 'most_popular_season' param
    # Secound, filter data with condition using 'most_popular_season'
    # Third, query the most shipping type
    query = """
        MATCH (u:User)-[p:PURCHASED]->(i:Item)
        WITH DISTINCT p.season as most_popular_season, COUNT(p.season) AS amount_season
        ORDER BY amount_season DESC
        LIMIT 1
        MATCH (u:User)-[p:PURCHASED]->(i:Item)
        WHERE p.season = most_popular_season
        WITH p.shipping_type AS shipping_type, COUNT(p) AS count, most_popular_season
        ORDER BY count DESC
        LIMIT 1
        RETURN most_popular_season as season, shipping_type as shipping;
    """
    result = connection.execute_query(query,None)
    print("The most popular season to shop in: ",result[0]['season'])
    print("The most popular shipping method in this season: ",result[0]['shipping'])

In [198]:
most_popular_season_and_shipping(neo4j_conn)

The most popular season to shop in:  Spring
The most popular shipping method in this season:  2-Day Shipping


Refer to q5.png

## Q6

In [204]:
# Collect the distinct locations
def get_distinct_location(connection):
    query = """
        MATCH (u:User)-[p:PURCHASED]->(i:Item)
        WITH DISTINCT u.location as location
        RETURN COLLECT(location) as list_location;
    """
    result = connection.execute_query(query,None)
    return result

In [206]:
locations = get_distinct_location(neo4j_conn)
print(locations[0]['list_location'])

['Kentucky', 'Oregon', 'Massachusetts', 'Maryland', 'Pennsylvania', 'South Carolina', 'West Virginia', 'Missouri', 'Tennessee', 'Georgia', 'Kansas', 'Utah', 'North Carolina', 'Virginia', 'New Hampshire', 'Alaska', 'Mississippi', 'Maine', 'California', 'Colorado', 'Wisconsin', 'Indiana', 'Ohio', 'Illinois', 'North Dakota', 'Rhode Island', 'New York', 'Hawaii', 'Michigan', 'Wyoming', 'New Mexico', 'Delaware', 'Minnesota', 'Connecticut', 'Arizona', 'New Jersey', 'Arkansas', 'Vermont', 'Idaho', 'Florida', 'Iowa', 'Nebraska', 'South Dakota', 'Alabama', 'Louisiana', 'Montana', 'Washington', 'Nevada', 'Oklahoma', 'Texas']


In [229]:
# Find the top 3 of popular items in each locations based on the amount of purchased
def top_3_items_at_location(connection,location):
    # query top 3 items sorted by the total purchased
    query = """
        MATCH (u:User)-[p:PURCHASED]->(i:Item)
        WHERE u.location = $location
        WITH u.location AS location, COUNT(i) AS count,i,p
        RETURN location, i.item_purchased as top_purchased_item, sum(p.purchase_amount) as sum_purchased
        ORDER BY sum_purchased DESC
        LIMIT 3;
    """
    parameter = {'location': str(location)}
    result = connection.execute_query(query,parameter)
    return result

In [230]:
# Print out the top 3 items for each location
for location in locations[0]['list_location']:
    print(location)
    top_item = top_3_items_at_location(neo4j_conn,location)
    items = [ item['top_purchased_item'] for item in top_item]
    print(items)
    print("-" * 30)

Kentucky
['Sweater', 'Handbag', 'Shirt']
------------------------------
Oregon
['Socks', 'Shirt', 'Blouse']
------------------------------
Massachusetts
['Dress', 'Boots', 'T-shirt']
------------------------------
Maryland
['Sunglasses', 'Sandals', 'Belt']
------------------------------
Pennsylvania
['Skirt', 'Scarf', 'Dress']
------------------------------
South Carolina
['Hoodie', 'Boots', 'Hat']
------------------------------
West Virginia
['Coat', 'Sunglasses', 'Handbag']
------------------------------
Missouri
['Sneakers', 'Sunglasses', 'Scarf']
------------------------------
Tennessee
['Scarf', 'Gloves', 'Skirt']
------------------------------
Georgia
['Dress', 'Blouse', 'Socks']
------------------------------
Kansas
['Sandals', 'Jewelry', 'Blouse']
------------------------------
Utah
['Shorts', 'Dress', 'Handbag']
------------------------------
North Carolina
['T-shirt', 'Gloves', 'Sweater']
------------------------------
Virginia
['Shoes', 'Shorts', 'Backpack']
----------------

In [231]:
# Print out the top 3 items for Kentucky
top_3_items_at_location(neo4j_conn,'Kentucky')

[{'location': 'Kentucky',
  'top_purchased_item': 'Sweater',
  'sum_purchased': 447},
 {'location': 'Kentucky',
  'top_purchased_item': 'Handbag',
  'sum_purchased': 382},
 {'location': 'Kentucky', 'top_purchased_item': 'Shirt', 'sum_purchased': 359}]

Refer to q6.png

In [238]:
# Close the connection when done
neo4j_conn.close()

Ciatation:
- Neo4j. (n.d.). Setting parameters in Cypher. Neo4j Cypher Manual. https://neo4j.com/docs/cypher-manual/current/syntax/parameters/
- Neo4j. (n.d.). Aggregating functions in Cypher. Neo4j Cypher Manual. https://neo4j.com/docs/cypher-manual/current/functions/aggregating/
- Farag, M. (2024). Lecture 5: NoSQL Database Labs: BigTable and Neo4J. Systems and Toolchains for AI Engineers.
- Farag, M. (2024). Lecture 4: NoSQL Databases. Systems and Toolchains for AI Engineers.
